# Import the necessary packages

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
import numpy
import tflearn
import tensorflow
import random
import json
import pickle
import csv

C:\Users\Kranthi Kumar\Anaconda3\envs\mp\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Kranthi Kumar\Anaconda3\envs\mp\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Kranthi Kumar\Anaconda3\envs\mp\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Kranthi Kumar\Anaconda3\envs\mp\lib\site-packages

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








# Load the dataset

In [3]:
with open("C:/Users/Kranthi Kumar/diseases.json") as file:
    data = json.load(file)

# Data pre-processing

In [4]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
            labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)

In [5]:
symbol="!@#$%^&*()_+={[}]'';:<>"
symptoms=["hi","hello","bye","thanks"]
with open("C:/Users/Kranthi Kumar/diseases1.csv") as file1:
    symp = csv.reader(file1, delimiter=',')
    for row in symp:
        for s in row:
            symptoms.append(s)

# Create a DNN model

In [6]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



# Fit and save the model

In [7]:
try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Kranthi Kumar\chatBotMajorProject_YT\model.tflearn


# Create function to give input to the Model.

In [8]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

# Create GUI Chat window and other necessary components with tkinter

In [9]:
#Creating GUI with tkinter
import tkinter
from tkinter import *

In [10]:
def chatbot_response(msg):
    results = model.predict([bag_of_words(msg, words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    if(results[results_index]>0.7):
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
                
        return random.choice(responses)
    else:
        return "Add some more info"


In [11]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg.isnumeric():
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res="You entered number.Please enter your symptoms"
        ChatLog.insert(END, "DocBot: " + res + '\n\n')
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
    elif msg.isalnum():
        flag=0
        for i in symptoms:
            if(msg.lower()==i.lower()):
                ChatLog.config(state=NORMAL)
                ChatLog.insert(END, "You: " + msg + '\n\n')
                ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
                res = chatbot_response(msg)
                ChatLog.insert(END, "DocBot: " + res + '\n\n')
                ChatLog.config(state=DISABLED)
                ChatLog.yview(END)
                flag=1
                break
        if(flag==0):
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "You: " + msg + '\n\n')
            ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
            res = "You entered something.Please enter your symptoms"
            ChatLog.insert(END, "DocBot: " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)

            
    if msg.lower()=="quit":
        base.destroy()

In [12]:
base = Tk()
base.title("DocBot")
base.geometry("400x500")
base.resizable(width=TRUE, height=TRUE)

''

In [13]:
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

In [14]:
base.mainloop()

In [15]:
import anvil.server
anvil.server.connect("XM46H6GNDBQU5ORWZILY6ISN-XCKWEJIILBT3FLCU")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [16]:
@anvil.server.callable


def DocBot(msg):
    if msg.isnumeric():
        return "You entered number.Please enter your symptoms"
    elif msg=="":
        return "you entered nothing.Please enter your symptoms"
    else:
        import nltk
        from nltk.stem.lancaster import LancasterStemmer
        stemmer = LancasterStemmer()
        def bag_of_words(s, words):
            bag = [0 for _ in range(len(words))]
            s_words = nltk.word_tokenize(s)
            s_words = [stemmer.stem(word.lower()) for word in s_words]
            for se in s_words:
                for i, w in enumerate(words):
                    if w == se:
                        bag[i] = 1
            return numpy.array(bag)

        results = model.predict([bag_of_words(msg, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        if(results[results_index]>0.7):
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            return random.choice(responses)
        else:
            return "Add some more info"    